### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
* If you would like to contribute to this notebook, make changes on it in useful_notebooks folder, run "Restart and Clear Output" before commit.

#### Unexpected Expansion Cases
###### Experiment set fetches unrealated labs and awards
Sometimes a fetch will get some unrelated labs and awards, this is because of the multiple awards a lab can have. This multiple awards are visited, which have users linked to them. This users also have labs, so here you go. Hopefully all are released/current already.

###### Experiment set fetches unrealted biosmaple/experiment/set
This was so far because of the experiment and biosample relation field, or the references field that links to a publication. If you ignore these fields, if should be fine.
`ignore_field = ['experiment_relation', 'biosample_relation', 'references']`

In [6]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

# Please Modify the following accordingly 
# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# Which status to change
change_status = 'released'

sets_in_scope = [] # ['4DNACCCC', '4DNACCCCC']
search_url = "/search/?experiments_in_set.experiment_type=dilution+Hi-C&experimentset_type=replicate&lab.display_title=Bing+Ren%2C+UCSD&status=pre-release&type=ExperimentSetReplicate"  # "/search/type=AAAAAAAA"


if sets_in_scope:
    set_to_release = [ff_utils.get_metadata(i, my_auth)['uuid'] for i in sets_in_scope]
elif search_url:
    set_to_release = [i['uuid'] for i in ff_utils.search_metadata(search_url, my_auth)]

store={}
item_uuids=[]

store, uuids = ff_utils.expand_es_metadata(set_to_release, my_auth, store_frame='object',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references'])

print(len(store['experiment_set_replicate']), 'exp sets for status change')
print(len(uuids), 'items collected')
time2 = time.time()
print(round((time2-time1), 1), 'sec for collection')

for i in store:
    print(i, len(store[i]))

10 exp sets for status change
1016 items collected
6.6 sec for collection
enzyme 1
workflow 13
software 12
individual_human 10
static_section 2
award 7
vendor 2
workflow_run_awsem 381
biosource 10
higlass_view_config 11
user 14
biosample 20
file_format 15
biosample_cell_culture 20
file_fastq 128
organism 1
experiment_hi_c 20
quality_metric_pairsqc 23
file_processed 152
ontology_term 24
experiment_set_replicate 10
ontology 2
file_reference 3
protocol 2
quality_metric_fastqc 128
lab 5


In [14]:
# TODO
# Check audits

# to decide which items to ignore as they have a 'higher' status
STATUS_LEVEL = {
    # standard_status
    "released": 10, "current": 10,
    "released to project": 9,
    "pre-release": 8,
    "planned": 6, "submission in progress": 6,
    "in review by lab": 4,
    "revoked": 0, "archived": 0,"deleted": 0, "obsolete": 0, "replaced": 0, "archived to project": 0,
    # additional file statuses
    'to be uploaded by workflow': 4, 'uploading': 4, 'uploaded': 4, 'upload failed': 4, 'draft': 4, 'released to lab': 4}

change_level = STATUS_LEVEL.get(change_status, 1)

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = [i for i in store if i.startswith('experiment') and not i.startswith('experiment_set')]
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(all_exps), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum'):
        print(a_file['accession'], 'missing content md5 sum')
    if not a_file.get('md5sum'):
        print(a_file['accession'], 'md5 was not calculated during upload, missing md5sum')
    if check_wfrs:
        pass
        # dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
if store.get('file_processed'):
    for a_file in store['file_processed']:
        if a_file['description'] == 'This is an intermediate file in the HiC processing pipeline':
            print(a_file['accession'], 'is from bwa-mem', a_file['date_created'][:16], end = "---")
            
#         if a_file['file_format'] == '/file-formats/pairs/':
#             if not a_file.get('quality_metric'):
#                 print(a_file['accession'], 'missing Pairsqc')
            if not a_file.get('source_experiments'):
                print('and has no source_experiment')
            else:
                print('is okay')
#             print(a_file['accession'], 'user submitted or produced by sbg runs')
#             print(a_file['description'], a_file['file_format'])
#         if check_wfrs:
#             pass
#             #dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
# list all wf types found
print('  Following run types are found:')
for wf in set([i['display_title'].split(' run')[0] for i in store.get('workflow_run_awsem')]):
           print('    ' + wf)
if store.get('workflow_run_awsem'):
    for wfr in store['workflow_run_awsem']:
        if wfr['run_status'] != 'complete':
            print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i, x['uuid'], x['status']] for x in store[i] if STATUS_LEVEL.get(x['status']) == 0]
    if weird:
        for case in weird:
            print(case)
        print()

EXPSET CHECK

EXP CHECK

FILE FASTQ CHECK

FILE PROCESSED CHECK
4DNFIXAHPX8O is from bwa-mem 2019-04-03T03:49---and has no source_experiment
4DNFIQLX8ML3 is from bwa-mem 2019-04-03T03:50---and has no source_experiment
4DNFIGR8VY1N is from bwa-mem 2019-04-03T03:51---is okay
4DNFIVBMB876 is from bwa-mem 2019-04-03T03:52---is okay
4DNFII8A81CH is from bwa-mem 2019-04-03T03:54---is okay
4DNFIDC612G8 is from bwa-mem 2019-04-03T03:52---is okay
4DNFIGNDJGGK is from bwa-mem 2019-04-03T03:51---is okay
4DNFIG6EF1Y9 is from bwa-mem 2019-04-03T03:52---is okay
4DNFIOURJS5S is from bwa-mem 2019-04-03T03:50---is okay
4DNFI22795UJ is from bwa-mem 2019-04-02T18:07---is okay
4DNFILNVV8V1 is from bwa-mem 2019-04-03T03:54---is okay
4DNFI3E7BB96 is from bwa-mem 2019-04-03T03:51---is okay
4DNFIE4Q1JSE is from bwa-mem 2019-04-03T03:54---is okay
4DNFIAUZ245Z is from bwa-mem 2019-04-03T03:51---is okay
4DNFI4Y9R7M4 is from bwa-mem 2019-04-03T03:51---is okay
4DNFIGSWDZKT is from bwa-mem 2019-04-03T03:51---is oka

In [20]:
for i in store:
    print(i, len(store[i]))
    weird = [x for x in store[i] if x['status'] == 'revoked']
    if weird:
        for case in weird:
            if case.get('static_content'):
                sc = [i for i in case['static_content'] if i.get('description', '').startswith('auto_generated')]
                if sc:
                    sc_uuid = sc[0]['content'].split('/')[2]
                    scit = [i for i in store['higlass_view_config'] if i['uuid'] == sc_uuid][0]
                    print(i, case['uuid'], sc_uuid, scit['status'])
                    ff_utils.patch_metadata({'status': 'released to lab'}, sc_uuid, my_auth)
            print(i, case['uuid'])
            ff_utils.patch_metadata({'status': 'pre-release'}, case['uuid'], my_auth)


file_processed 113
file_processed fd9a037b-ddc9-4086-8339-10163fb6121d 3ef3d359-b3be-48a1-9167-5f056f4161e0 released
file_processed fd9a037b-ddc9-4086-8339-10163fb6121d
file_processed f37b66d0-fc4a-4490-95b3-674548f53972
file_processed e74550c2-aa57-4ffb-8ef1-3b7227ca8cf9 f1c2eab3-63ab-4382-b673-ab51bb578277 released
file_processed e74550c2-aa57-4ffb-8ef1-3b7227ca8cf9
file_processed e275e5bf-f44b-4d68-bf3d-3fb24bd81023
file_processed e0d4f71b-249b-4065-b004-879e630fc91c
file_processed dbfd8f87-7463-4fe1-9c56-5d5612780073 23a715d3-ee99-4c66-8e18-c2f54c610501 released
file_processed dbfd8f87-7463-4fe1-9c56-5d5612780073
file_processed da45ae30-7faa-44c9-9563-f3b6ce73fd5c
file_processed c774e6d0-d972-4c94-b425-d90dd6b12dfc
file_processed c0d02802-b6e0-44b9-ad3b-43354060f559 3ee877f2-d3d6-463f-9fe5-79595c31edc6 released
file_processed c0d02802-b6e0-44b9-ad3b-43354060f559
file_processed bb028d51-4a59-47eb-a560-8939d3cb2115
file_processed b1b54589-4231-4c8d-a992-25cd10e50e8d 02c4fc13-5e76-40d

file_processed 4bda2040-65ef-46cf-a414-c57b46f44cd7 15679e74-f992-44c3-9c62-0598a1bac641 released
file_processed 4bda2040-65ef-46cf-a414-c57b46f44cd7
file_processed 48fa96d3-1ff5-4630-8828-d118867b7474
file_processed 43fc38af-d363-4804-a9c4-4112a2fd7ee3
file_processed 3fe1e603-4c38-47bb-8544-149f702f1ac9
file_processed 2bf0b821-5d22-42d4-8109-1dcde8165a50
file_processed 27e81a96-741c-47ce-8527-64cb58263505
file_processed 25af07f2-8136-4375-8209-c01d136c6574 73b4e3a8-75c5-4bd4-92dd-a6ab3d080827 released
file_processed 25af07f2-8136-4375-8209-c01d136c6574
file_processed 1a964e05-c66e-4664-a843-690c01acb3f7 958af260-a1bc-4bde-ac3f-e399e03656ae released
file_processed 1a964e05-c66e-4664-a843-690c01acb3f7
lab 5
ontology_term 15
static_section 4
biosource 1
protocol 26
biosample 11
file_format 10
award 4
ontology 1
badge 1
antibody 8
file_fastq 44
experiment_set_replicate 28
quality_metric_dedupqc_repliseq 44
quality_metric_dedupqc_repliseq d2745e28-8536-450b-9fbb-e9d0ff92fbed
quality_metric

In [ ]:
# Check status
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} needs change'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))
    
       

In [ ]:
# If you want to patch the status, change action to True
action = False

reviewed = ""
reviewed = input('Did another wrangler review this release? (y/n):')
if reviewed != 'y':
    raise KeyError('A key step is missing!')
    
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if action:
                patch_data = {'status': change_status}
                if change_status == 'released' and a_type in ['publication']:
                    patch_data = {'status': 'current'}
                   
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth) 

        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} UPDATED with status'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))